# <span class="mark">Preprocessing data</span>

In [1]:
import numpy as np # numpy
from random import randint # data set을 만들기 위한 random library
from sklearn.preprocessing import MinMaxScaler # data normaliza를 위한 scale 라이브러리

### 데이터 예제
- 13 ~ 65세를 위한 어떤 약을 만들기 위해 임상실험 진행
- 2000명의 표본으로 반은 65세 이하를 대상으로 나머지 반은 65세 이상으로 진행
- 95%의 확률로 65세 이상은 부작용 발생
- 95%의 확률로 65세 미만은 부작용 미발생

In [2]:
# X_data : train 데이터, Y_data : label
# 13 ~ 64세는 95%는 부작용 미발생, 5% 발생
# 65세 이상(100세 이하)는 95%는 부작용 발생, 5%는 미발생
# 2000개의 데이터 = 950(13 ~64 : 0) + 50(13 ~ 64 : 1) + 950(65 ~ 100 : 1) + 50(65 ~ 100 : 0)

# Train data 생성, 나이는 13 ~ 64세로 랜덤하게 만들며 1000개의 샘플을 만듭니다.
X_data = np.random.randint(low = 13, high = 64, size = 1000) 
# Test data 생성, 나이는 13 ~ 64세이고 95%는 무작용 미발생, 5%는 발생하도록 만듭니다.
Y_data = np.zeros(950) # 13 ~ 64세 95%는 부작용 미발생
Y_data = np.append(Y_data, np.ones(50)) # 13 ~ 64 95%는 부작용 발생

# Training data 생성, 나이는 65~100세로 랜덤하게 만들며 1000개의 샘플을 만듭니다.
X_data = np.append(X_data, np.random.randint(low = 65, high = 100, size = 1000))
# test data 생성, 나이는 65 ~ 100세이고, 95%는 부작용 발생, 5%는 부작용 미발생하도록 만듭니다.
Y_data = np.append(Y_data, np.ones(950)) # 65 ~ 100세 95%는 부작용 발생
Y_data = np.append(Y_data, np.zeros(50)) # 65 ~ 100세 5%는 부작용 미발생

In [3]:
# Input data scale : 너무 큰 input 값은 weight가 수렴하는 데 오래 걸려
# 학습하는 데 시간이 오래 걸리므로 data scale을 통하여 값을 줄여주어 빠르게 학습하도록 합니다.

# [0, 1] 사이의 값으로 스케일 변화를 위한 변수 생성
scalar = MinMaxScaler(feature_range= (0, 1)) 
# (None, 1) shape으로 [0,1] 범위로 스케일 변경
scaled_X_data = scalar.fit_transform(X_data.reshape(-1, 1)) 

C:\Users\infoe\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [4]:
print("X_data : ", X_data) # Train data
print("X_data.shape : ", X_data.shape) # Train data shape
print("scaled_X_data : ", scaled_X_data) # scaled train data
print(scaled_X_data.shape) # scaled train data shape
print(Y_data) # Label data
print(Y_data.shape) # label data shape

X_data :  [19 52 28 ... 89 78 69]
X_data.shape :  (2000,)
scaled_X_data :  [[0.06976744]
 [0.45348837]
 [0.1744186 ]
 ...
 [0.88372093]
 [0.75581395]
 [0.65116279]]
(2000, 1)
[0. 0. 0. ... 0. 0. 0.]
(2000,)


# <span class="mark">Neural Network</span>

In [5]:
import keras 
from keras import backend as K 
from keras.models import Sequential # Sequential : model 선언
from keras.layers import Activation # Activation 사용
from keras.layers.core import Dense # Dense layer 사용
from keras.optimizers import Adam # Adam optimizer 사용
from keras.metrics import categorical_crossentropy # cross entropy 사용

C:\Users\infoe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1. Sequential을 선언한 후 layer를 쌓습니다.

In [6]:
# Sequential()을 이용하여 model을 선언합니다.
model = Sequential()
# model의 전체 틀은 model.summary()를 이용하여 확인하면 보기 편합니다.

# Dense layer를 추가합니다. 1-dim vector → (None, 16)
# activation으로 ReLU를 사용합니다.
model.add(Dense(16, input_shape=(1,), activation = 'relu'))
# Dense layer를 추가합니다. → (None, 32)
# activation으로 ReLU를 사용합니다.
model.add(Dense(32, activation='relu'))
# Dense layer를 추가합니다. → (None, 2)
# 출력층이므로 activation으로 softmax를 사용합니다.
model.add(Dense(2, activation = 'softmax'))

### 2. 쌓은 layer의 정보를 얻습니다. shape 및 parameter

In [27]:
# 각 layer의 output shape과 Parameter의 수를 요약해서 볼 수 있습니다.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 16)                32        
_________________________________________________________________
dense_8 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [28]:
# model.compile을 이용하여 optimizer와 loss를 선언합니다. 
# optimizer(학습 방법) : Adam 
# loss function(loss 계산 방법) : sparse_categorical_cross_entropy (one-hot 아닐 때는 sparse 써야한다.)
# metrics(performance 판단 기준) : accuracy
model.compile(optimizer = Adam(lr = 0.01),
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [29]:
# model.fit을 통하여 학습을 합니다.
# input data : scaled_X_data 
# output data : Y_data
# validation ratio : 0.2
# batch size : 10
# epochs : 20
# shuffle (epoch 마다 batch 순서가 shuffle 되는 지 유무) : True (default : True)

model.fit(scaled_X_data, Y_data, validation_split = 0.2, batch_size=16, epochs=20, shuffle=True)

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 1s 345us/step - loss: 0.3313 - acc: 0.9075 - val_loss: 0.5730 - val_acc: 0.8750
Epoch 2/20
1600/1600 [==============================] - 0s 206us/step - loss: 0.1752 - acc: 0.9500 - val_loss: 1.0194 - val_acc: 0.8750
Epoch 3/20
1600/1600 [==============================] - 0s 202us/step - loss: 0.1556 - acc: 0.9581 - val_loss: 1.2400 - val_acc: 0.8750
Epoch 4/20
1600/1600 [==============================] - 0s 214us/step - loss: 0.1579 - acc: 0.9506 - val_loss: 1.2253 - val_acc: 0.8425
Epoch 5/20
1600/1600 [==============================] - 0s 196us/step - loss: 0.1599 - acc: 0.9519 - val_loss: 1.3554 - val_acc: 0.8750
Epoch 6/20
1600/1600 [==============================] - 0s 212us/step - loss: 0.1502 - acc: 0.9550 - val_loss: 1.4979 - val_acc: 0.8750
Epoch 7/20
1600/1600 [==============================] - 0s 207us/step - loss: 0.1498 - acc: 0.9600 - val_loss: 1.6314 - val_acc: 0.8750
E

# Predict

In [33]:
# model.predict를 이용하여 input에 대한 결과를 예측합니다.
# input shape과 동일하게 맞추어서 데이터를 입력하면 됩니다.
# 예제에서는 학습하였던 데이터를 재사용해서 입력해 보겠습니다.

# model.predict를 사용하면 각 클래스의 확률을 출력합니다.
predictions = model.predict(scaled_X_data, batch_size=16)
print(predictions)

[[9.4078732e-01 5.9212748e-02]
 [9.4078749e-01 5.9212506e-02]
 [9.0622270e-01 9.3777336e-02]
 ...
 [1.6088597e-21 1.0000000e+00]
 [1.5854526e-07 9.9999988e-01]
 [3.4541810e-16 1.0000000e+00]]


In [32]:
# model.predict_classes를 사용하면 각 클래스 확률을 비교하여 가장 큰 확률을 클래스를 출력합니다.
# 위의 model.predict의 첫 3개는 0 클래스의 확률이 높고 끝 3개는 1 클래스 확률이 높습니다.
# 따라서 아래 결과의 첫 3개는 0 클래스이고 끝 3개는 1 클래스입니다.
class_predictions = model.predict_classes(scaled_X_data, batch_size=16)
print(class_predictions)

[0 0 0 ... 1 1 1]
